In [13]:
import pandas as pd
import sqlite3
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

db_path = 'data.db'
connection = sqlite3.connect(db_path)

queryTrain = "SELECT * FROM teams WHERE year < 10 ORDER BY year"
train_data = pd.read_sql_query(queryTrain, connection)

queryTest = "SELECT * FROM teams WHERE year = 10"
test_data = pd.read_sql_query(queryTest, connection)

# Define your features (X) and target (y) for training data
X_train = train_data.drop(columns=['playoff', 'lgID', 'tmID', 'franchID','confID','divID', 'seeded', 'semis','finals','name', 'firstRound', 'arena'])  # Exclude non-feature columns
y_train = train_data['playoff']  # Your target variable

# Define your features (X) and target (y) for test data
X_test = test_data.drop(columns=['playoff', 'lgID', 'tmID', 'franchID','confID','divID', 'seeded', 'semis','finals','name', 'firstRound', 'arena'])  # Exclude non-feature columns
y_test = test_data['playoff']  # Your target variable

# Create a Decision Tree classifier
clf = DecisionTreeClassifier(random_state=0)

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate accuracy on the test data
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)





Accuracy: 1.0


In [ ]:
header = data.iloc[0]
data = data[1:]  # Skip the header row

# Separate the last row for testing
test_data = data.iloc[-1:]
train_data = data.iloc[:-1]

# Define the column you want to use as the target variable
target_column = 'playoff'  # Replace with your column name

# Set the header as the column names for the DataFrame
train_data.columns = header
test_data.columns = header

# Split the data into features (X) and the target variable (y)
X_train = train_data.drop(columns=[target_column])
y_train = train_data[target_column]
X_test = test_data.drop(columns=[target_column])
y_test = test_data[target_column]

# Perform one-hot encoding on categorical columns with explicit categories
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse=False, drop='first', categories='auto')
X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[categorical_columns]))
X_test_encoded = pd.DataFrame(encoder.transform(X_test[categorical_columns]))

# Reset the index of the encoded DataFrames to match the original data
X_train_encoded.index = X_train.index
X_test_encoded.index = X_test.index

# Drop the original categorical columns and concatenate the encoded ones
X_train = X_train.drop(columns=categorical_columns)
X_test = X_test.drop(columns=categorical_columns)

X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

# Create and train a Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model using accuracy or other metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")